In [ ]:
%pip install torch torchvision torchaudio  xformers==0.0.20  

In [ ]:
import os
HOME = "/notebooks"
print(HOME)

In [ ]:
%cd {HOME}
!git clone https://github.com/IDEA-Research/GroundingDINO.git
%cd {HOME}/GroundingDINO
%pip install -q -e .
%pip install -q roboflow

In [ ]:
%pip install git+https://github.com/facebookresearch/segment-anything.git


In [ ]:
%pip install opencv-python pycocotools matplotlib onnxruntime onnx

In [ ]:
import os
import torch
from torchvision.ops import box_convert

WEIGHTS_NAME = "groundingdino_swint_ogc.pth"
WEIGHTS_PATH = "/notebooks/ComfyUI/models/grounding-dino/groundingdino_swint_ogc.pth"
print(WEIGHTS_PATH, "; exist:", os.path.isfile(WEIGHTS_PATH))
CONFIG_PATH = os.path.join(HOME, "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py")
print(CONFIG_PATH, "; exist:", os.path.isfile(CONFIG_PATH))

In [ ]:
# Set the input and output directories
input_dir = '/notebooks/lora_training/Blockout/blockout/to_crop'
output_dir = '/notebooks/lora_training/Blockout/blockout/cropped'

# Set device to cpu for Mac M1
device = "cuda"
# Load the SAM model
sam_checkpoint = "/notebooks/ComfyUI/models/sams/sam_vit_h_4b8939.pth"
model_type = "vit_h"

In [ ]:
%pip install ultralytics

In [ ]:

from groundingdino.util.inference import load_model, load_image, predict, annotate

model = load_model(CONFIG_PATH, WEIGHTS_PATH)

In [ ]:
TEXT_PROMPT = "character"
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25


In [ ]:

for filename in os.listdir(input_dir):
    if filename.endswith('.txt'):
        continue  # Skip to the next iteration if the file is a .txt file
    # Load the image using Pillow
    image_path = os.path.join(input_dir, filename)
    try:
        image_pil = Image.open(image_path)
    except IOError:
        print(f"Cannot open {filename}. Skipping...")
        continue 

    # Convert the Pillow Image object to a NumPy array (as the mask generator expects a NumPy array)
    image_np = np.array(image_pil)
    
    # Generate masks for the image
    image_source, image = load_image(image_path)

    boxes, logits, phrases = predict(
        model=model, 
        image=image, 
        caption=TEXT_PROMPT, 
        box_threshold=BOX_TRESHOLD, 
        text_threshold=TEXT_TRESHOLD
    )

  
    image_width, image_height = image_pil.size
    # Convert the normalized cxcywh coordinates to pixel xyxy coordinates
    boxes_pixel_coords = box_convert(boxes=boxes, in_fmt="cxcywh", out_fmt="xyxy") * torch.Tensor([image_width, image_height, image_width, image_height])

    for i, box in enumerate(boxes_pixel_coords):
        # The box_convert function returns a tensor with the same dtype as the input.
        # Convert the tensor to a numpy array and then to int for use with Image.crop.
        box = box.numpy().astype(int)

        # Unpack the box coordinates
        x_min, y_min, x_max, y_max = box
          # Convert the normalized bounding box coordinates to pixel coordinates
        cropped_image_pil = image_pil.crop((x_min, y_min, x_max, y_max))
        output_path = os.path.join(output_dir, f'{filename.split(".")[0]}_crop{i}.png')
        cropped_image_pil.save(output_path)

In [ ]:
!rm /notebooks/lora_training/Blockout/blockout/cropped/* -rf

In [ ]:
%cd /notebooks

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
# Set up the environment
%pip install basicsr
%pip install facexlib
%pip install gfpgan
%pip install -r requirements.txt
%run setup.py develop